In [ ]:
#panel serve PythonJavascriptCommunications.ipynb --dev

In [1]:
import holoviews as hv; hv.extension('bokeh', 'plotly', logo=None)
import panel as pn;     pn.extension('katex')

from panel.interact import interact

#import scipy.io
import numpy as np

<div style="float:center;width:100%;text-align: center;">
    <strong style="height:60px;color:darkred;font-size:40px;">Integrate Python With Javascript</strong><br>
</div>

# 1. Javascript Sends Array to Python

In [33]:
import param
import panel as pn
from panel.custom import JSComponent

pn.extension()

class RandomArrayGenerator(JSComponent):
    """
    A custom JSComponent that generates two arrays of random floats and sends them to Python.
    """

    # Parameter to store the generated array
    value = param.String(default="")

    _esm = """
export function render({model, el}) {
    console.log("RandomArrayGenerator initialized.");

    // Create a container to display generated arrays
    const output = document.createElement('div');
    output.style.padding = "10px";
    output.style.border = "1px solid #ddd";
    output.style.marginTop = "10px";
    output.style.fontFamily = "monospace";
    output.style.background = "#f9f9f9";
    output.textContent = "Waiting for data...";
    el.appendChild(output);

    // Track the number of updates
    let count = 0;

    // Function to generate a random array of floats
    const generateArray = () => {
        if (count >= 2) {
            clearInterval(interval); // Stop after sending two arrays
            console.log("Stopped after sending two arrays.");
            return;
        }

        count += 1;

        const randomArray = Array.from({length: 10}, () => Math.random().toFixed(2));
        console.log("Generated Array:", randomArray);

        // Update the display
        output.textContent = "Generated Array: " + JSON.stringify(randomArray);

        // Notify Python by updating the `value` parameter
        model.value = JSON.stringify(randomArray);
    };

    // Start generating random arrays every 3 seconds
    const interval = setInterval(generateArray, 3000);

    // Cleanup when the component is removed
    model.on('change', (changed) => {
        if (changed.value === undefined) {
            clearInterval(interval);
            console.log("RandomArrayGenerator stopped.");
        }
    });
}
"""

# Instantiate the custom JSComponent
random_array_generator = RandomArrayGenerator()

# Widget to display the received array in Python
data_display = pn.widgets.StaticText(name="Received Array", value="Waiting for data...")

# Python callback to process the data when `value` changes
@pn.depends(random_array_generator.param.value, watch=True)
def update_display(value):
    import json
    try:
        array = json.loads(value)
        formatted = ", ".join(f"{float(x):.2f}" for x in array)
        data_display.value = f"Received Array: {formatted}"
    except (json.JSONDecodeError, ValueError):
        data_display.value = "Invalid data received"

# Layout of the Panel app
layout = pn.Column(
    pn.pane.Markdown("### Random Float Array Generator (Sends Two Arrays Only)"),
    pn.Row(
        pn.Column(
            "# JavaScript generates two random arrays and stops.",
            random_array_generator
        ),
        pn.Column(
            "# Python receives and displays the array:",
            data_display
        )
    )
)

# Serve the app
layout.servable()


Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] RandomArrayGenerator()
        [1] Column
            [0] Markdown(str)
            [1] StaticText(name='Received Array', value='Waiting for data...')

# 2. Python Sends Array to Two Javascript Loggers

In [36]:
import param
import panel as pn
from panel.custom import JSComponent

pn.extension()

class ArrayLogger(JSComponent):
    """
    A custom JSComponent where Python sends an array, and JavaScript logs it to the console.
    Each instance is uniquely identified.
    """

    # Explicitly define parameters
    array = param.List(default=[])  # Array to send to JavaScript
    logger_id = param.String(default="")  # Unique ID for the logger instance

    _esm = """
export function render({model, el}) {
    console.log(`ArrayLogger initialized (ID: ${model.logger_id})`);

    // Create a display area in the DOM
    const output = document.createElement('div');
    output.style.padding = "10px";
    output.style.border = "1px solid #ddd";
    output.style.marginTop = "10px";
    output.style.fontFamily = "monospace";
    output.style.background = "#f9f9f9";
    output.textContent = `Logger ${model.logger_id}: No array received yet.`;
    el.appendChild(output);

    // Watch for changes in the 'array' parameter
    model.on('change:array', () => {
        const receivedArray = model.array;
        console.log(`Logger ${model.logger_id} - Received array:`, receivedArray);

        // Update the display
        if (Array.isArray(receivedArray)) {
            output.textContent = `Logger ${model.logger_id}: Array Received: ` + JSON.stringify(receivedArray);
        } else {
            output.textContent = `Logger ${model.logger_id}: Invalid array received.`;
        }
    });
}
"""

# Instantiate two ArrayLogger components with unique IDs
array_logger_1 = ArrayLogger(logger_id="1")
array_logger_2 = ArrayLogger(logger_id="2")

# Buttons to trigger sending arrays to each logger
send_button_1 = pn.widgets.Button(name="Send Array to JS - Logger 1", button_type="primary")
send_button_2 = pn.widgets.Button(name="Send Array to JS - Logger 2", button_type="primary")

# Callbacks to send arrays to JavaScript
def send_array_to_js_1(event):
    example_array = [1.23, 4.56, 7.89, 10.11, 12.34]
    array_logger_1.array = example_array
    print("Sent array to Logger 1:", example_array)

def send_array_to_js_2(event):
    example_array = [9.87, 6.54, 3.21, 0.98, 0.12]
    array_logger_2.array = example_array
    print("Sent array to Logger 2:", example_array)

# Attach callbacks to buttons
send_button_1.on_click(send_array_to_js_1)
send_button_2.on_click(send_array_to_js_2)

# Layout to display the app
layout = pn.Column(
    pn.pane.Markdown("### Python Sends Arrays to Two Independent JavaScript Loggers"),
    pn.Row(
        pn.Column(
            "#### Logger 1",
            send_button_1,
            array_logger_1
        ),
        pn.Column(
            "#### Logger 2",
            send_button_2,
            array_logger_2
        )
    )
)

# Serve the app
layout.servable()



Sent array to Logger 1: [1.23, 4.56, 7.89, 10.11, 12.34] 
Sent array to Logger 2: [9.87, 6.54, 3.21, 0.98, 0.12] 
Sent array to Logger 2: [9.87, 6.54, 3.21, 0.98, 0.12] 
Sent array to Logger 2: [9.87, 6.54, 3.21, 0.98, 0.12]

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] Button(button_type='primary', name='Send Array to J...)
            [2] ArrayLogger(logger_id='1')
        [1] Column
            [0] Markdown(str)
            [1] Button(button_type='primary', name='Send Array to J...)
            [2] ArrayLogger(logger_id='2')

# 3. Draw Moving Objects with P5.js

In [28]:
import panel as pn
import param

from panel.custom import JSComponent

class P5Sketch(JSComponent):

    num_components = param.Integer(default=5, bounds=(1,15))

    _esm = """
    import p5 from "https://esm.sh/p5"

    export function render({model, el}) {
      function sketch(p) {
        let time = 0;
        let wave = [];

        let run = true;

        p.setup = () => {
          const canvas = p.createCanvas(model.width, model.height)
          canvas.canvas.style.visibility = 'visible'
          canvas.mousePressed(() => {
            if (p.mouseX >= 0 && p.mouseX <= p.width && p.mouseY >= 0 && p.mouseY <= p.height) {
              run = !run;
              if (run) { p.loop(); } else { p.noLoop(); }
            }
          }
        )};

        p.draw = () => {
          p.background(0);
          p.translate(150, 200);

          let x = 0;
          let y = 0;

          const n = model.num_components;

          for (let i = 0; i < n; i++) {
            let prevx = x;
            let prevy = y;

            let n = i * 2 + 1;
            let radius = 75 * (4 / (n * Math.PI));
            x += radius * Math.cos(n * time);
            y += radius * Math.sin(n * time);

            p.stroke(255, 100);
            p.noFill();
            p.ellipse(prevx, prevy, radius * 2, radius * 2);
            p.stroke(255);
            p.line(prevx, prevy, x, y);
          }
          wave.unshift(y);
          p.translate(200, 0);
          p.line(x - 200, y, 0, wave[0]);
          p.beginShape();
              p.noFill();
              for (let i = 0; i < wave.length; i++) {
                p.vertex(i, wave[i] );
              }
          p.endShape();

          time += 0.05;

          if (wave.length > 250) { wave.pop(); }
        }
      }
      new p5(sketch, el)
    }
    """

sketch = P5Sketch(width=600, height=400)

pn.Column("# Fourier Demo", "Mouse Press starts/stops the animation", sketch.controls(['num_components']), sketch).servable()

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] WidgetBox(margin=(5, 10), name='Controls')
        [0] StaticText(value='<b>Controls</b>')
        [1] IntSlider(end=15, name='Num components', start=1, value=5)
    [3] P5Sketch(height=400, sizing_mode='fixed', width=600)

# 4. Geometrical Construction with JSXgraph

In [38]:
from panel.custom import JSComponent
import panel as pn

class JSXGraphComponent(JSComponent):
    _esm = r"""
    import JXG from 'https://cdn.jsdelivr.net/npm/jsxgraph/distrib/jsxgraphcore.mjs';
    import TEX from 'https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js';

    export function render({ model, el }) {
      // Create a div element to hold the JSXGraph board
      let boardDiv          = document.createElement("div");
      boardDiv.style.width  = "300px";
      boardDiv.style.height = "300px";
      el.appendChild(boardDiv);

      // Global JSXGraph options for rendering
      JXG.Options.label.autoPosition = true;
      JXG.Options.text.useMathJax    = true;
      JXG.Options.text.fontSize      = 20;

      // =========== Initialize the JSXGraph board ======================================
      let board = JXG.JSXGraph.initBoard(boardDiv, {
        boundingbox: [-5, 5, 5, -5],
        showCopyright:  false, showNavigation: false, axis: false });

      // =========== Add points =========================================================
      const p = board.create('point', [-3, -3], { name: 'p' });
      const q = board.create('point', [ 0,  2], { name: 'q' });
      const r = board.create('point', [ 3, -3], { name: 'r' });

      // =========== Add labeled arrows (vectors) with proper label configuration =======
      board.create('arrow', [p, q], { withLabel: true, name: '\\(\\vec{pq}\\)',
        label: { position: 'top', parse: false }, lastArrow: { type: 4, size: 8 }});

      board.create('arrow', [q, r], { withLabel: true, name: '\\(\\vec{qr}\\)',
        label: { position: 'top', parse: false }, lastArrow: { type: 4, size: 8 } });

      board.create('arrow', [p, r], { withLabel: true, name: '\\(\\vec{pr}\\)',
        label: { position: 'top', parse: false }, lastArrow: { type: 4, size: 8 } });

      // =========== Cleanup when the component is removed ==============================
      model.on('remove', () => {
        console.log("JSXGraphComponent removed.");
        JXG.JSXGraph.freeBoard(board);
      });
    }
    """

pn.extension()
pn.Column(pn.Row( JSXGraphComponent(), JSXGraphComponent(),width=320, height=320),
).servable()


Column
    [0] Row(height=320, sizing_mode='fixed', width=320)
        [0] JSXGraphComponent()
        [1] JSXGraphComponent()